In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [11]:
chars = sorted(list(set(''.join(words))))
stoi = { s: i + 1 for i, s in enumerate(chars) }
stoi['.'] = 0
itos = { i: s for s, i in stoi.items() }
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [12]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return X, Y

In [13]:
import random
random.seed(42)

In [15]:
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr,  Ytr  = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte,  Yte  = build_dataset(words[n2:])

In [26]:
# Our previous MLP implementation
vocab_size = len(stoi)
n_embed = 10 # the dimensions for character embedding in a vector
n_hidden = 200 # number of neurons in the hidden layer

g  = torch.Generator().manual_seed(2147483647)
C  = torch.randn((vocab_size, n_embed),    generator = g) # our initial lookup table (an n_embed dimensional embedding)
W1 = torch.randn((n_embed * block_size, n_hidden),      generator = g) # our hidden layer matrix
b1 = torch.randn(n_hidden,                 generator = g) # our biases 
W2 = torch.randn((n_hidden, vocab_size),   generator = g) # our output layer
b2 = torch.randn(vocab_size,               generator = g)

parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

11897


In [27]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    
    # create our mini-batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y
    
    # forward pass
    emb = C[Xb]                         # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concat the vectors
    hpreact = embcat @ W1 + b1          # hidden layer pre-activation
    h = torch.tanh(hpreact)
    
    logits = h @ W2 + b2                # output layer
    loss = F.cross_entropy(logits, Yb)  # loss function
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    lr = 0.1 if i < 1000000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 200000: 24.9158
  10000/ 200000: 2.4926
  20000/ 200000: 2.4397
  30000/ 200000: 2.7249
  40000/ 200000: 2.2882
  50000/ 200000: 2.3702
  60000/ 200000: 3.0447
  70000/ 200000: 2.2345
  80000/ 200000: 2.2451
  90000/ 200000: 2.0264
 100000/ 200000: 2.2131
 110000/ 200000: 1.7773
 120000/ 200000: 2.1576
 130000/ 200000: 2.0001
 140000/ 200000: 2.0393
 150000/ 200000: 2.4155
 160000/ 200000: 2.0013
 170000/ 200000: 2.2775
 180000/ 200000: 2.3017
 190000/ 200000: 2.5306


In [28]:
@torch.no_grad() # avoid gradient tracking here (no back-propagation)
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val':   (Xdev, Ydev),
        'test':  (Xte, Yte)
    }[split]
    
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1) # concat the vectors
    hpreact = embcat @ W1 + b1          # hidden layer pre-activation
    h = torch.tanh(hpreact)
    
    logits = h @ W2 + b2                # output layer
    loss = F.cross_entropy(logits, y)  # loss function
    print(split, loss.item())
    
split_loss('train')
split_loss('val')

train 2.2171318531036377
val 2.275200605392456


In [30]:
# sample from our model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size
    while True:
        
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        
        context = context[1:] + [ix]
        out.append(ix)
        
        if ix == 0:
            break
            
    print("".join(itos[i] for i in out))

carmah.
aar.
hari.
kimla.
geaty.
halassa.
jazon.
namperric.
igeric.
eliana.
chaiiv.
asleigh.
ham.
poi.
demi.
tia.
lilah.
vid.
isi.
athon.
